<a href="https://colab.research.google.com/github/Ivalomat/4-elements/blob/main/Docking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem
import os
import subprocess

# Cargar base de datos
df = pd.read_csv("Enamine10k_scores.csv")
ligand_dir = "ligands"
os.makedirs(ligand_dir, exist_ok=True)

for idx, row in df.iterrows():
    smiles = row["SMILES"]
    mol_id = f"mol_{idx:04d}"
    mol = Chem.MolFromSmiles(smiles)
    mol = Chem.AddHs(mol)
    AllChem.EmbedMolecule(mol)
    AllChem.UFFOptimizeMolecule(mol)

    sdf_path = os.path.join(ligand_dir, f"{mol_id}.sdf")
    pdbqt_path = os.path.join(ligand_dir, f"{mol_id}.pdbqt")

    # Guardar como SDF
    writer = Chem.SDWriter(sdf_path)
    writer.write(mol)
    writer.close()

    # Convertir a PDBQT usando Open Babel
    subprocess.run(["obabel", sdf_path, "-O", pdbqt_path, "--gen3d"])

print("Conversión a PDBQT completada.")

In [ ]:
import os
import subprocess

# Parámetros del docking según el artículo
CENTER_X = 33.19
CENTER_Y = 1.06
CENTER_Z = 15.62
SIZE_X = 30
SIZE_Y = 30
SIZE_Z = 30
EXHAUSTIVENESS = 8
NUM_MODES = 9
ENERGY_RANGE = 3

RECEPTOR = "receptor.pdbqt"
LIGANDS_DIR = "ligands"
OUTPUT_DIR = "results"
os.makedirs(OUTPUT_DIR, exist_ok=True)

for ligand_file in os.listdir(LIGANDS_DIR):
    if ligand_file.endswith(".pdbqt"):
        ligand_path = os.path.join(LIGANDS_DIR, ligand_file)
        out_path = os.path.join(OUTPUT_DIR, f"{ligand_file.replace('.pdbqt', '')}_out.pdbqt")
        log_path = os.path.join(OUTPUT_DIR, f"{ligand_file.replace('.pdbqt', '')}_log.txt")

        command = [
            "vina",
            "--receptor", RECEPTOR,
            "--ligand", ligand_path,
            "--center_x", str(CENTER_X),
            "--center_y", str(CENTER_Y),
            "--center_z", str(CENTER_Z),
            "--size_x", str(SIZE_X),
            "--size_y", str(SIZE_Y),
            "--size_z", str(SIZE_Z),
            "--out", out_path,
            "--log", log_path,
            "--exhaustiveness", str(EXHAUSTIVENESS),
            "--num_modes", str(NUM_MODES),
            "--energy_range", str(ENERGY_RANGE)
        ]

        print(f"Docking {ligand_file} ...")
        subprocess.run(command)

print("Docking terminado.")

In [ ]:
import pandas as pd
import os
import re

# Cargar resultados originales
df = pd.read_csv("Enamine10k_scores.csv")
my_scores = []

for idx in range(len(df)):
    mol_id = f"mol_{idx:04d}"
    log_file = f"results/{mol_id}_log.txt"
    affinity = None
    if os.path.exists(log_file):
        with open(log_file, 'r') as f:
            for line in f:
                if "REMARK VINA RESULT" in line:
                    affinity = float(re.findall(r"-?\\d+\\.\\d+", line)[0])
                    break
    my_scores.append(affinity)

df["My Docking score (kcal/mol)"] = my_scores
df.to_csv("docking_comparison.csv", index=False)
print("✅ Archivo comparativo generado: docking_comparison.csv")